# Kaggle VS Code Remote Setup
Run all cells to set up remote VS Code connection via zrok

In [ ]:
# Configuration - EDIT THESE VALUES
ZROK_TOKEN = "YOUR_ZROK_TOKEN"  # Get from https://zrok.io
SSH_PASSWORD = "kaggle123"  # Set your SSH password
SSH_PUBLIC_KEY = ""  # Optional: Your SSH public key for key-based auth

In [ ]:
# Cell 1: Install system dependencies
!apt-get update -qq
!apt-get install -y -qq openssh-server > /dev/null 2>&1
print("✅ SSH server installed")

In [ ]:
# Cell 2: Configure SSH
import subprocess

# Set root password
subprocess.run(f"echo 'root:{SSH_PASSWORD}' | chpasswd", shell=True)

# Configure SSH to allow root login
!sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
!sed -i 's/#PasswordAuthentication yes/PasswordAuthentication yes/' /etc/ssh/sshd_config

# Add SSH public key if provided
if SSH_PUBLIC_KEY:
    !mkdir -p /root/.ssh
    !echo "{SSH_PUBLIC_KEY}" >> /root/.ssh/authorized_keys
    !chmod 700 /root/.ssh
    !chmod 600 /root/.ssh/authorized_keys
    print("✅ SSH key added")

# Start SSH service
!service ssh start
print("✅ SSH server started")

In [ ]:
# Cell 3: Install zrok
!wget -q https://github.com/openziti/zrok/releases/download/v0.4.44/zrok_0.4.44_linux_amd64.tar.gz
!tar -xzf zrok_0.4.44_linux_amd64.tar.gz
!chmod +x zrok
!mv zrok /usr/local/bin/
print("✅ zrok installed")

In [ ]:
# Cell 4: Enable zrok with your token
!zrok enable {ZROK_TOKEN}
print("✅ zrok enabled")

In [ ]:
# Cell 5: Start zrok tunnel (this will display connection info)
import subprocess
import threading
import time
import re

def run_zrok():
    process = subprocess.Popen(
        ['zrok', 'share', 'public', '--backend-mode', 'tcpTunnel', 'localhost:22'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    for line in process.stdout:
        print(line, end='')
        if 'https://' in line:
            url = re.search(r'https://[\w.-]+', line)
            if url:
                print(f"\n" + "="*60)
                print("🎉 CONNECTION INFO:")
                print(f"="*60)
                print(f"Tunnel URL: {url.group()}")
                print(f"SSH Command: ssh root@{url.group().replace('https://', '')} -p 22")
                print(f"Password: {SSH_PASSWORD}")
                print("="*60 + "\n")

thread = threading.Thread(target=run_zrok, daemon=True)
thread.start()
print("🚀 Starting zrok tunnel... Please wait for connection info.")

In [ ]:
# Cell 6: Keep notebook alive
import time
print("⏰ Keeping notebook alive. Don't close this tab!")
print("Press Ctrl+C to stop.")

while True:
    time.sleep(60)
    print(f"Still running... {time.strftime('%H:%M:%S')}")